## TransMAP Example: AIS data 

Using the Transmap API to work with Moving Pandas

### API Docs 
- https://oak.cast.uark.edu/docs

### Requirements
- must be on UARK VPN to work (for now)



In [ ]:
# Source ref: https://github.com/anitagraser/movingpandas-examples/blob/main/analysis-examples/2-ship-data.ipynb

make_url = lambda uri: "https://oak.cast.uark.edu/{0}".format(uri)

In [ ]:
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import requests
import numpy as np
import pandas as pd
import geopandas
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

In [ ]:
mpd.__version__

In [ ]:
ais_data = requests.get(make_url('ais/vessels'), params={
    "limit": 100000
}).json()

In [ ]:
df = pd.DataFrame(ais_data)
df2 = pd.DataFrame(df.location.tolist(), index= df.index)
gpdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df2[0], df2[1]))

In [ ]:
gpdf.set_crs('epsg:4326')

In [ ]:
gpdf.plot()

In [ ]:
gpdf['t'] = pd.to_datetime(gpdf['BaseDateTime'], format='%Y-%m-%dT%H:%M:%S')

In [ ]:
gpdf = gpdf.set_index('t')

In [ ]:
gpdf['SOG'].hist(bins=10, figsize=(15, 3))

In [ ]:
print("Original size: {} rows".format(len(gpdf)))
gpdf = gpdf[gpdf.SOG>0]
print("Reduced to {} rows after removing 0 speed records".format(len(gpdf)))
gpdf['SOG'].hist(bins=10, figsize=(15,3))

In [ ]:
gpdf['VesselType'] = gpdf['VesselType'].astype(float)
gpdf['VesselType'].value_counts().plot(kind='bar', figsize=(15,3))

In [ ]:
%%time
MIN_LENGTH = 0.02 # degrees - May need to reproject to use meters
traj_collection = mpd.TrajectoryCollection(gpdf, 'MMSI', min_length=MIN_LENGTH)
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
traj_collection = mpd.MinTimeDeltaGeneralizer(traj_collection).generalize(tolerance=timedelta(minutes=1))
# this is looking for a specific 't' index

In [ ]:
shiptype_to_color = {'Passenger': 'blue', 'HSC': 'green', 'Tanker': 'red', 'Cargo': 'orange'}
# Should be changed to map to these codes: https://coast.noaa.gov/data/marinecadastre/ais/VesselTypeCodes2018.pdf
traj_collection.plot(column='VesselType', column_to_color=shiptype_to_color, linewidth=1, capstyle='round')

In [ ]:
passenger = traj_collection
passenger.hvplot(title='Passenger ferries', line_width=2)